In [1]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd
import requests
import pymongo
from flask import Flask
#flask_pymongo import is generating an error code. Pip install Flask-Pymongo
##documentation here: https://flask-pymongo.readthedocs.io/en/latest/
from flask_pymongo import PyMongo
from flask import render_template, redirect
###pip install fixed the error.

In [2]:
#Set up the executable path and the browser. Boiler plate code.
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False) 

In [3]:
#link to the url from the assignment readme
redplanet_url = 'https://redplanetscience.com/'
browser.visit(redplanet_url)

In [4]:
#create the html, soup, and do the scraping
html = browser.html

redplanet_soup = bs(html, 'html.parser')

#assign as variables the scraped data
news_title = redplanet_soup.find_all('div', class_='content_title')[0].text
news_paragraph = redplanet_soup.find_all('div', class_='article_teaser_body')[0].text

print(news_title)
print("=================================================")
print(news_paragraph)

NASA's Curiosity Mars Rover Takes a New Selfie Before Record Climb
Along with capturing an image before its steepest ascent ever, the robotic explorer filmed its "selfie stick," or robotic arm, in action.


In [5]:
# JPL Mars Space Images - Featured Image
##Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).
###Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.
####Make sure to find the image url to the full size `.jpg` image.
#####Make sure to save a complete url string for this image.

jpl_image_url = 'https://spaceimages-mars.com'
image_url = 'https://spaceimages-mars.com/image/?search=&category=Mars'

browser.visit(image_url)

#browser.find_by_name('featured').click()

html = browser.html

image_soup = bs(html, 'html.parser')

image_path = [i.get('src') for i in image_soup.find_all('img', class_='headerimage fade-in')]
image_path[0]
              


IndexError: list index out of range

In [ ]:
featured_image_url = jpl_image_url + image_path[0]
featured_image_url

In [ ]:
#Mars Facts
##Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
###Use Pandas to convert the data to a HTML table string.

facts_url = 'https://galaxyfacts-mars.com'
browser.visit(facts_url)


In [ ]:
#Mars Facts Pandas Portion
facts_df = pd.read_html(facts_url)[1]
facts_df.columns = ['Description', 'Mars']
facts_df

In [ ]:
#Mars Facts pandas dataframe to html portion
##transpose the df. Documentation: https://numpy.org/doc/stable/reference/generated/numpy.matrix.transpose.html

facts_transpose = facts_df.transpose()
facts_transpose

In [ ]:
#take the transposed dataframe and render in in html with .to_html
facts_html = facts_transpose.to_html('MarsFacts.html', index=False)
facts_html

In [8]:
#Mars Hemispheres
#Visit the astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of Mar's hemispheres.
##You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
###Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. 
####Use a Python dictionary to store the data using the keys `img_url` and `title`.
####Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

#####First set up the URLs and paths and soup

hemi_url = 'https://marshemispheres.com/'
hemi_url_path = requests.get(hemi_url)

hemi_soup = bs(hemi_url_path.content)


In [9]:
#Next find the div items for the images
hemi_div_item = hemi_soup.find_all('div', class_='item')

In [10]:
#Next find the links to the images and append them to a dictionary
hemi_links = []
for i in hemi_div_item:
    for a in i.find_all('a', href=True):
        if a.text:
            hemi_links.append(a['href'])
#print(hemi_links)

In [11]:
#NEXT take the dictionary of individual links and combine it with the url for the hemishperes
full_hemi_links = [hemi_url + i for i in hemi_links]
full_hemi_links

['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']

In [12]:
def extract_image(url):
    response = requests.get(url)
    soup = bs(response.text, "html.parser")
    image = soup.find_all("img", class_="wide-image")
    img = [i.get("src") for i in image]
    return "https://marshemispheres.com/" + img[0]

In [13]:
hemi_image_url = [extract_image(i) for i in full_hemi_links]
hemi_image_url

['https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [14]:
#reading the instructions, i am supposed to be using img_url for that run, so I am redoing it and renaming it
img_url = [extract_image(i) for i in full_hemi_links]
img_url

['https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [15]:
#again, following the instructions, need to perform a similar operation for the title of the images
def image_title(url):
    response = requests.get(url)
    soup = bs(response.text, "html.parser")
    title = soup.find_all("h2", class_="title")
    title_text = [i.text for i in title]
    return title_text[0]

In [17]:
#now output the title code
title = [image_title(i) for i in full_hemi_links]
title

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [18]:
#now we can create a list of tuples with the zip function to combine the img_url and titles we just acquired
url_list = list(zip(title, img_url))
url_list

[('Cerberus Hemisphere Enhanced',
  'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'),
 ('Schiaparelli Hemisphere Enhanced',
  'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'),
 ('Syrtis Major Hemisphere Enhanced',
  'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'),
 ('Valles Marineris Hemisphere Enhanced',
  'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg')]

In [19]:
#call a function to combine the total urls as requested in the HW ReadME

keys = ('title', 'img_url')

def url_function(keys, url_list):
    url_dictionary = [dict(zip(keys, values)) for values in url_list]
    return url_dictionary

In [20]:
#call the output and display
complete_hemi_urls = url_function(keys, url_list)
complete_hemi_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]